# Tutorial 4: Molecular Fingerprints
분자는 여러 가지 방법으로 표현될 수 있다. 이 튜토리얼에서는 "분자 지문(molecular fingerprint)"이라고 하는 표현 유형을 소개합니다. 그것은 종종 작은 약물과 같은 분자들에게 잘 작용하는 매우 간단한 표현이다.

In [1]:
import deepchem as dc
dc.__version__

'2.5.0'

## What is a Fingerprint?

딥 러닝 모델은 거의 항상 숫자 배열을 입력으로 사용한다. 우리가 분자들을 처리하려면, 우리는 어떻게든 각각의 분자들을 하나 이상의 숫자의 배열로 나타낼 필요가 있습니다.

많은 모델 유형(전부는 아니지만)은 고정된 크기를 입력하도록 요구한다. 이것은 분자들에게 어려운 일이 될 수 있는데, 분자들은 다른 수의 원자를 가지고 있기 때문이다. 이러한 유형의 모델을 사용하려면 고정된 크기의 배열로 가변 크기의 분자를 표현해야 한다.

fingerprint은 이러한 문제를 해결하기 위해 고안되었다. 지문은 고정된 길이 배열로, 다른 요소들이 분자에 다른 특징들의 존재를 나타낸다. 만약 두 분자가 유사한 fingerprints을 가지고 있다면, 그것은 그것들이 많은 동일한 특징들을 포함하고 있다는 것을 나타내며, 따라서 유사한 화학작용을 가질 가능성이 있다.

DeepChem은 "Extended Connectivity Fingerprint" 또는 줄여서 "ECFP"라고 불리는 특정한 종류의 fingerprint을 지원한다. 그것들은 또한 때때로 "circular fingerprints"이라고 불립니다. ECFP 알고리즘은 원자의 직접적인 특성과 결합만을 기준으로 분류하는 것으로 시작한다. 각각의 고유한 패턴이 특징입니다. 예를 들어, "두 개의 수소와 두 개의 무거운 원자에 결합된 탄소 원자"가 특징이며, fingerprint의 특정 요소는 그 특징을 포함하는 분자에 대해 1로 설정됩니다. 그런 다음 더 큰 원형 인접 지역을 확인하여 새로운 특징을 반복적으로 식별합니다. 다른 두 가지 특정 형상에 결합된 하나의 특정 형상은 더 높은 수준의 형상이 되며, 해당 원소는 그것을 포함하는 모든 분자에 대해 설정된다. 이는 고정된 횟수(대개 2회) 동안 계속됩니다.

이제 ECFP로 특징지어진 데이터 세트를 살펴보겠습니다.

In [2]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP')
train_dataset, valid_dataset, test_dataset = datasets
print(train_dataset)

<DiskDataset X.shape: (6264, 1024), y.shape: (6264, 12), w.shape: (6264, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>


The feature array `X` shape = (6264, 1024). 이는 training set에 6264개의 샘플이 있다는 것을 의미합니다. 각각의 fingerprint은 1024 길이로 표현된다. 레이블 배열 `y`에 shape(6264, 12): multitask dataset입니다. Tox21은 분자의 독성에 대한 정보를 포함하고 있다. 12개의 다른 분석들이 독성의 징후를 찾는데 사용되었다. 데이터 세트는 12개 모든 검사의 결과를 각각 다른 작업으로 기록합니다.

또한 가중치 배열을 살펴봅시다.

In [3]:
train_dataset.w

array([[1.04502242, 1.03632599, 1.12502653, ..., 1.05576503, 1.17464996,
        1.05288369],
       [1.04502242, 1.03632599, 1.12502653, ..., 1.05576503, 1.17464996,
        1.05288369],
       [1.04502242, 1.03632599, 1.12502653, ..., 1.05576503, 0.        ,
        1.05288369],
       ...,
       [1.04502242, 0.        , 1.12502653, ..., 1.05576503, 6.7257384 ,
        1.05288369],
       [1.04502242, 1.03632599, 1.12502653, ..., 1.05576503, 6.7257384 ,
        1.05288369],
       [1.04502242, 1.03632599, 1.12502653, ..., 0.        , 1.17464996,
        1.05288369]])

일부 요소는 0이다. weights는 손실 데이터를 나타내는 데 사용되고 있다. 모든 분석이 모든 분자에 대해 실제로 수행된 것은 아닙니다. sample 또는 sample/task pair의 가중치를 0으로 설정하면 적합 및 평가 중에 무시됩니다. 이는 손실 함수나 기타 메트릭스에 영향을 미치지 않습니다.

대부분의 다른 가중치는 1에 가깝지만 정확히 1은 아닙니다. 이 작업은 각 작업에 대한 양성 및 음성 검체의 전체 가중치의 균형을 맞추기 위해 수행됩니다. 모델을 훈련할 때, 우리는 12가지 과제 각각이 동등하게 기여하기를 바라며, 각 과제에서 양성 샘플과 음성 샘플에 동일한 가중치를 부여하기를 원한다. 그렇지 않으면, 모델은 대부분의 훈련 샘플이 무독성이라는 것을 알게 되고(Imbalance), 따라서 다른 분자를 무독성으로 식별하는 데 치우치게 될 수 있다.

## Training a Model on Fingerprints
모델을 훈련시키자. 이전 튜토리얼에서는 `GraphConvModel`을 사용했는데, 이것은 복잡한 입력 집합을 취하는 상당히 복잡한 아키텍처이다. 지문은 단일 고정된 길이 배열로 매우 간단하기 때문에 우리는 훨씬 더 간단한 유형의 모델을 사용할 수 있다.

In [4]:
model = dc.models.MultitaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000])

`MultitaskClassifier`은 fully connected layers의 simple stack입니다. 이 예에서는 폭 1000의 single hidden layer를 사용하도록 지시한다. 우리는 또한 각 입력에 1024개의 기능이 있을 것이며 12개의 다른 작업에 대한 예측을 생성해야 한다고 말한다.

각 작업에 대해 별도의 모델을 train하는 것은 어떨까요? 그럴 수도 있지만, 여러 작업에 대해 단일 모델을 교육하는 것이 종종 더 효과적이라는 것이 밝혀졌습니다. 우리는 나중에 튜토리얼에서 그것의 예를 볼 것이다.

모델을 교육하고 평가해 봅시다.

In [6]:
import numpy as np

model.fit(train_dataset, nb_epoch=10)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('training set score: ', model.evaluate(train_dataset, [metric], transformers))
print('test set score: ', model.evaluate(test_dataset, [metric], transformers))

training set score:  {'roc_auc_score': 0.9574440829187995}
test set score:  {'roc_auc_score': 0.6819259082250904}


간단한 model 및 featurization에 대한 성능도 나쁘지 않습니다. 이 데이터 집합에서는 보다 정교한 모델이 약간 더 낫지만, 훨씬 더 나은 것은 아니다.